In [2]:
import sumolib 
import traci
from sumolib import checkBinary

In [3]:
junction = {'KlongTei': 'cluster_1892287670_272491964_272492178',
      'Rama4': 'cluster_272488163_282390730_66263210_66263222',
      'NaRanong': 'cluster_272488164_272492179_3457051443_61907354',
      'Sunthon': 'gneJ83',
      'KasemRat': 'cluster_272448137_272555800_272555808_7660045934_7710268409',
      'Virtual' : '270329335'}
direction = {'KlongTei' : ['NB','WB','EB'], 
             'Rama4': ['WB','SB','EB','NB'],
             'NaRanong': ['NB','WB','SW','SB','EB'],
             'Sunthon': ['WB','SB','EB','NB'],
             'KasemRat': ['WB','EB','NB'],
             'Virtual' : ['SB','EB']}

In [4]:
# KlongTei = 'cluster_1892287670_272491964_272492178'
# Rama4 = 'cluster_272488163_282390730_66263210_66263222'
# NaRanong = 'cluster_272488164_272492179_3457051443_61907354'
# Sunthon = 'gneJ83'
# KasemRat = 'cluster_272448137_272555800_272555808_7660045934_7710268409'
# Virtual = '270329335'
# junction = [KlongTei,Rama4,NaRanong,Sunthon,KasemRat,Virtual]

In [5]:
def reset():
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "Kasemrat-SUMO-UsingBookNetFile/osm.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1'])

In [6]:
net1 = sumolib.net.readNet('Kasemrat-SUMO-UsingBookNetFile/AddLane.net.xml')

In [16]:
def get_connect(ne):
    dir_con = set()
    out = ne.getToNode().getOutgoing()
    for o in out:
#                                 print(o)
        con = sumolib.net.edge.Edge.getConnections(ne,o)
#                                 print(con)
        for c in con:
            dir_con.add(sumolib.net.connection.Connection.getDirection(c).upper())
    connect = ''
    if 'L' in dir_con : connect += 'L'
    else: connect += 'X'
    if 'S' in dir_con : connect += 'S'
    else: connect += 'X'
    if 'R' in dir_con : connect += 'R'
    else: connect += 'X'
    if 'T' in dir_con : connect += 'U'
    else: connect += 'X'
    return connect

In [8]:
reset()

In [152]:
traci.close()

In [21]:
lane_name = {}
last_edge = None
NoNeed = ['27702347#6', '300813736#0', '825786399','459492913#3', '459492913#2','459492913#0','618739073#1',
          '825786398', '482384842#2', '751759907']
for j in junction.keys(): 
    if j == 'Sunthon':
        dir_index = -1
        EdgeIndex = 1
        edge_1 = list(traci.trafficlight.getControlledLanes(junction[j]))
        for e_1 in edge_1:
            if e_1[-1] == '0' and e_1[:-2] != last_edge:
                EdgeIndex = 1
                dir_index += 1
                edge = e_1[:-2]
                for i in range(10):
                    nextEdges = net1.getEdge(edge).getFromNode().getIncoming()
                    for ne in nextEdges:
                        if ne.getLaneNumber() > 1 and ne.getID() not in NoNeed:
                            connect = get_connect(ne)
    #                         print(connect)
                            EdgeIndex += 1
                            edgeID = ne
                            numOfLane = edgeID.getLaneNumber()
                            for LaneIndex in range(numOfLane):
                                nextLanes = edgeID.getID()+'_'+str(LaneIndex)
                                if 'R' in connect and LaneIndex == numOfLane-1 and 'U' not in connect:
                                    lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+'XSRX'] = nextLanes
                                elif 'R' in connect and LaneIndex == numOfLane-1 and 'U' in connect:
                                    lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+'XSRU'] = nextLanes
                                elif 'L' in connect and LaneIndex == 0:
                                    lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+'LSXX'] = nextLanes
                                else:
                                    lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(EdgeIndex)+'_'+str(LaneIndex)+'_'+'XSXX'] = nextLanes
                            edge = edgeID.getID()
            LaneIndex = int(e_1[-1])
            connect = get_connect(net1.getEdge(e_1[:-2]))
#             print(connect)
            numOfLane = net1.getEdge(e_1[:-2]).getLaneNumber()
            if 'R' in connect and LaneIndex == numOfLane-1 and 'U' not in connect:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSRX'] = e_1
            elif 'R' in connect and LaneIndex == numOfLane-1 and 'U' in connect:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSRU'] = e_1
            elif 'L' in connect and LaneIndex == 0:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'LSXX'] = e_1
            else:
                lane_name[str(j).upper()+'_'+direction[j][dir_index]+'_'+str(1)+'_'+str(LaneIndex)+'_'+'XSXX'] = e_1
            last_edge = e_1[:-2]
for l in sorted(lane_name):
# for l in lane_name:
    print(l, '=', "'"+lane_name[l]+"'")

SUNTHON_EB_1_0_LSXX = '824116555#3_0'
SUNTHON_EB_1_1_XSXX = '824116555#3_1'
SUNTHON_EB_1_2_XSRU = '824116555#3_2'
SUNTHON_EB_2_0_XSXX = '824116555#0-AddedOffRampEdge_0'
SUNTHON_EB_2_1_XSXX = '824116555#0-AddedOffRampEdge_1'
SUNTHON_EB_2_2_XSXX = '824116555#0-AddedOffRampEdge_2'
SUNTHON_EB_2_3_XSRX = '824116555#0-AddedOffRampEdge_3'
SUNTHON_EB_3_0_XSXX = '824116555#0_0'
SUNTHON_EB_3_1_XSXX = '824116555#0_1'
SUNTHON_EB_3_2_XSXX = '824116555#0_2'
SUNTHON_EB_4_0_XSXX = '824116554_0'
SUNTHON_EB_4_1_XSXX = '824116554_1'
SUNTHON_NB_1_0_LSXX = '153225687#13_0'
SUNTHON_NB_1_1_XSXX = '153225687#13_1'
SUNTHON_NB_1_2_XSRX = '153225687#13_2'
SUNTHON_NB_2_0_LSXX = '153225687#10-AddedOffRampEdge_0'
SUNTHON_NB_2_1_XSXX = '153225687#10-AddedOffRampEdge_1'
SUNTHON_NB_2_2_XSXX = '153225687#10-AddedOffRampEdge_2'
SUNTHON_NB_3_0_XSXX = '153225687#10_0'
SUNTHON_NB_3_1_XSXX = '153225687#10_1'
SUNTHON_NB_3_2_XSXX = '153225687#10_2'
SUNTHON_NB_4_0_LSXX = '153225687#9-AddedOnRampEdge_0'
SUNTHON_NB_4_1_XSXX = '1